# **Machine Learning** 
Machine learning: is a branch of AI (artificial intelligence) and computer science that focuses on using data and algorithms to mimic how humans learn, gradually improving its accuracy. <a href="https://www.ibm.com/it-it/cloud/learn/machine-learning?utm_content=SRCWW&p1=Search&p4=43700068096860752&p5=e&gclid=Cj0KCQiAq5meBhCyARIsAJrtdr7E2ZBLXuBKgWyGOKpIswvC-8dzOiWJvZpfGW6XbDBZ2E0f9oVEpFYaAjigEALw_wcB&gclsrc=aw.ds">IBM</a>

### **Computer Vision** 
Computer vision: is a field of artificial intelligence that trains computers to interpret and  understand the visual world.Using digital images from cameras and videos and deep learning models,machines can accurately identify and classify objects — and then react to what they “see.” <a href="https://www.sas.com/en_us/insights/analytics/computer-vision.html#:~:text=Computer%20vision%20is%20a%20field,to%20what%20they%20%E2%80%9Csee.%E2%80%9D">SAS<a>


<img src="https://d1m75rqqgidzqn.cloudfront.net/2019/10/shutterstock_158795060.jpg" width="600" height="150" alt="Computer vision">





### **Feature_Descriptors**
Feature_Descriptors: 
A feature descriptor is a method that extracts the feature descriptions for an interest point (or the full image). Feature descriptors serve as a kind of numerical “fingerprint” that we can use to distinguish one feature from another by encoding interesting information into a string of numbers. <a href="https://www.baeldung.com/cs/image-processing-feature-descriptors">Baeldung</a>



### **Task**
Image classification of <a href="https://www.cs.toronto.edu/~kriz/cifar.html">CIFAR10</a> dataset using Machine Learning classifiers and Feature Descriptors

<img src="https://production-media.paperswithcode.com/datasets/4fdf2b82-2bc3-4f97-ba51-400322b228b1.png">

### Part 0 - Importing the libraries
Let's start importing some libraries.
In particular, `sklearn` is the library for the **Machine Learning stuff**!

In [ ]:
import sklearn
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from glob import glob
from os.path import join
import cv2
from skimage.feature import hog, local_binary_pattern
from tqdm import tqdm

# import tensorflow and keras just to get the cifar10 dataset
import tensorflow as tf
from tensorflow import keras

The seed is important to have **deterministic** experiments.

The `seed()` method is used to initialize the random number generator.
The random number generator needs a number to start with (a seed value), to be able to generate a random number.

In [ ]:
np.random.seed(1821)

### Part 1 - Functions and Classes
`get_labels()` is a function that receives a name (`string`) and returns the class (`int`), following this:

*   airplane:   0        
*   automobile: 1
*   bird:       2
*   cat:        3
*   deer:       4
*   dog:        5
*   frog:       6
*   horse:      7
*   ship:       8
*   truck:      9

In [ ]:
# CIFAR -10
def get_labels(name):
    if 'airplane' in name:
        return 0
    elif 'automobile' in name:
        return 1
    elif 'bird' in name:
        return 2
    elif 'cat' in name:
        return 3
    elif 'deer' in name:
        return 4
    elif 'dog' in name:
        return 5
    elif 'frog' in name:
        return 6
    elif 'horse' in name:
        return 7
    elif 'ship' in name:
        return 8
    elif 'truck' in name:
        return 9                                    
    else:
        raise NotImplementedError('Not existing class!')

`extract_feature()` is a function that, given a list of images, compute a Feature Descriptor.
We are going to use two libraries: `opencv` to handle images (opening, resizing) and `skimage` to compute features.

Specifically:
* `feat_type=1` → function computes HOG
* `feat_type=2` → function computes LBP
* `feat_type=3` → function does not computes any feature, but simply unroll the input image

**Tools**:
*   `cv2.imread()`: open an image (0: gray level, 1: BGR)
*   `cv2.resize()`: resize an image
*   `hog()`: compute HOG feature
*   `lbp()`: compute LBP feature

Be aware that the feature computation time (and the final accuracy of the model) are strictly related to the image size!


In [ ]:
def extract_features(images, feat_type, img_size):

    labels = []
    features = []

    for image in tqdm(images):

        # open the image use 1 for BGR or colored images/ 3 channel
        img = cv2.imread(image, 1)

        # resize the image
        img = cv2.resize(img, (img_size, img_size))

        # compute the features
        if feat_type == 'hog':
            feat = hog(img, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(1, 1))
        elif feat_type == 'lbp':
            feat = np.ravel(local_binary_pattern(img, P=100, R=5))
        elif feat_type == 'img':
            img = img / 255.0
            feat = np.ravel(img)
        else:
            raise NotImplementedError('Not implemented feature!')

        # append features and labels
        features.append(feat)
        labels.append(get_labels(image))

    return features, labels

### Part 2 - Load Data


1.   Upload the `.zip` file containing the *Cifar10* dataset
2.   Unzip the file using the following comand. Dataset folders will appear in `/content`



In [ ]:
!unzip -q cifar10.zip -d /content

Let's create a list of all images available in the dataset.

**Tools**:
* `join()`: joins one or more path components intelligently. The return value is the concatenation of path.
* `glob()`: returns a possibly empty list of path names that match names. Wildcards (in that case `*` are allowed!)

In [ ]:
dataset_path_train = './train'
images_train = glob(join(dataset_path_train, '*', '*.png'))
print('Images: ', len(images_train))

Images:  50000


In [ ]:
dataset_path_test = './test'
images_test = glob(join(dataset_path_test, '*', '*.png'))
print('Images: ', len(images_test ))

Images:  10000


In [ ]:
type(images_test)

list

### Part 3.1 - Splitting the data into Training, Validation and testing

in this case it is essential to have a **training**, **validation** and **test** sets. 

Training data are used to train the model, while the validation split is used to assess performance.

Test final accuracy of our model.

We put **70% of data in training**, **30% in validation** from the training dataset folder, and the  **100% of unseen testing dataset from the test folder**.

In [ ]:
# shuffle both test and train images extracted from the zip folder
np.random.shuffle(images_train)
np.random.shuffle(images_test)

# split and assign 70% of the data to trainset
trainset = images_train[:int(0.7*len(images_train))]
# 30% of the data to valset
valset = images_train[int(0.7*len(images_train)):]
# test assign to previous loaded images_test
testset = images_test
length = valset + trainset
print('Total: {} splitted in Train: {}, Val: {} and Test: {}'.format(len(images_test + images_train), len(trainset), len(valset), len(testset)))

Total: 60000 splitted in Train: 35000, Val: 15000 and Test: 10000


Here, we define two important elements: the **size of the images** (used to compute feature descriptors) and the **type of features**.
We use a **progress bar** (`tqdm`) to show the state of the feature computation!

In [ ]:
# Analyze also the computational load, in terms of time, of the solution
import time

st = time.time()

In [ ]:
# classifierAlgorithm + 128x128 + HOG
img_size = 128
feature_type = 'hog'

train_x, train_y = extract_features(trainset, feature_type, img_size)
val_x, val_y = extract_features(valset, feature_type, img_size)
test_x, test_y = extract_features(testset, feature_type, img_size)

100%|██████████| 10000/10000 [04:28<00:00, 37.24it/s]


### Classifier
Now that the feature extraction has ended, we can define our classifiers.
As in the previous case, we start our analysis from the SVM,.

In [ ]:
# suppor vector machine (kernel svm)
clf = svm.SVC(gamma=0.001, C=100, kernel='rbf', verbose=False, random_state=0)

### Training
Now we are ready for the training!
With `sklearn` library is tremendously simple, we just need training data (`train_x` and the related labels `train_y`) and pass them to the classifier.

**Tools**:
-   `model.fit()`: fit the provided model with training data.

In [ ]:
clf.fit(train_x, train_y)

SVC(C=100, gamma=0.001, random_state=0)

### Validation

In [ ]:
clf.score(val_x, val_y)

0.5701333333333334

### Testing
Now we are reading to use our classifier! The trained classifier output the labels (as defined above) for the classification task.

Tools:
  - `model.predict()`: predict the class of the given data.

In [ ]:
y_pred = clf.predict(test_x)
print('Predicted {} samples: {}'.format(len(y_pred), y_pred))

Predicted 10000 samples: [6 4 7 ... 3 5 0]


In [ ]:
# pair prediction and ground truth
for i in zip(y_pred,test_y):
  print(i, end=" ")

(6, 2) (4, 3) (7, 9) (7, 3) (8, 0) (5, 3) (0, 0) (6, 6) (0, 0) (0, 5) (7, 7) (5, 2) (0, 8) (6, 6) (3, 3) (6, 4) (1, 0) (8, 9) (1, 1) (5, 4) (4, 7) (3, 7) (6, 6) (3, 3) (7, 7) (4, 4) (8, 9) (8, 9) (5, 5) (1, 1) (8, 8) (7, 7) (0, 1) (6, 0) (9, 9) (7, 5) (1, 1) (3, 4) (6, 6) (8, 0) (8, 8) (8, 8) (8, 9) (9, 9) (2, 2) (6, 6) (1, 1) (9, 7) (0, 0) (0, 1) (2, 2) (5, 4) (2, 4) (1, 7) (8, 9) (5, 3) (3, 3) (9, 9) (3, 5) (8, 8) (9, 9) (6, 6) (6, 6) (1, 2) (0, 0) (2, 2) (7, 0) (5, 5) (3, 4) (0, 0) (8, 4) (9, 8) (3, 5) (9, 9) (3, 5) (9, 9) (4, 4) (0, 0) (1, 1) (5, 5) (2, 3) (9, 9) (8, 8) (4, 2) (0, 0) (2, 2) (3, 3) (6, 6) (0, 0) (4, 4) (9, 9) (9, 9) (3, 5) (4, 4) (3, 5) (3, 3) (1, 1) (7, 4) (5, 7) (9, 9) (2, 2) (8, 8) (9, 9) (8, 8) (0, 0) (4, 7) (2, 0) (4, 7) (0, 0) (3, 2) (0, 2) (6, 6) (6, 6) (9, 8) (0, 0) (2, 2) (5, 5) (2, 2) (6, 6) (0, 0) (1, 1) (1, 1) (7, 9) (1, 1) (3, 3) (9, 9) (7, 7) (6, 6) (0, 0) (3, 3) (9, 9) (1, 1) (2, 2) (9, 9) (4, 2) (4, 8) (0, 2) (2, 5) (0, 3) (0, 2) (0, 2) (6, 6) (8, 0)

In [ ]:
# Error rate
from logging import logProcesses
Error = round(1 - accuracy_score(test_y, y_pred),3)
print(Error)

0.428


It's time to understand the final performance of the trained classifier.

**Tools**:
   * `accuracy_score()`: Accuracy classification score. The set of labels predicted for a sample must exactly match the corresponding set of labels of GT.

In [ ]:
from logging import logProcesses
print('Final Accuracy: {:.3f}'.format(accuracy_score(test_y, y_pred)))

Final Accuracy: 0.572


In [ ]:
# Analyze also the computational load, in terms of time, of the solution
# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')
print('Execution time:', elapsed_time/60, 'minutes')
print('Execution time:', elapsed_time/(60*60), 'hours')

Execution time: 16922.37468314171 seconds
Execution time: 282.03957805236183 minutes
Execution time: 4.70065963420603 hours


### Confusion matrix

We can also compute the confusion matrix to further understand the performance on the trained model.

**Tools**:
   * `confusion_matrix()`: computes confusion matrix to evaluate the accuracy of a classification.
   * `plot_confusion_matrix()`: plots Confusion Matrix (it is deprectaed and will be removed in future versions of the library).

In [ ]:
# compute the confusion matrix
c_matrix = confusion_matrix(test_y, y_pred)
print(c_matrix)
# print(type(c_matrix))
# confusion matrix with normalized values
print("\n", c_matrix.diagonal() / c_matrix.sum(axis=1))

[[629  28  83  28  33  22  13  10 129  25]
 [ 30 774  10  22  15   7  25   4  56  57]
 [ 99  12 376 110 151 100  62  43  39   8]
 [ 57  25  92 374  86 173  95  43  25  30]
 [ 36  13 119  76 492  63  96  73  25   7]
 [ 19  12  85 205  69 467  53  67  14   9]
 [ 23  14  59  73  61  54 680  11  17   8]
 [ 25   9  55  59 105  80   8 612  24  23]
 [126  83  33  19  25  10  23  19 620  42]
 [ 28  88  15  25  20  19  10  40  63 692]]

 [0.629 0.774 0.376 0.374 0.492 0.467 0.68  0.612 0.62  0.692]
